In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np 
from scipy import signal 
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa
from keras import regularizers

2024-02-09 21:46:18.825215: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-09 21:46:19.055270: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 21:46:19.055324: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 21:46:19.055370: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-09 21:46:19.111117: I tensorflow/core/platform/cpu_feature_g

In [3]:
x_train = np.load('Datasets/GAT/traindata.npy',mmap_mode='r')
y_train = np.load('Datasets/GAT/trainlabels.npy',mmap_mode='r')
x_test  = np.load('Datasets/GAT/testdata.npy',mmap_mode='r')
y_test  = np.load('Datasets/GAT/testlabels.npy',mmap_mode='r')

In [4]:
mean=x_train.mean()
std=x_train.std()

x_train=(x_train-mean)/std
x_test=(x_test-mean)/std

x_train=np.expand_dims(x_train,axis=-1)
x_test=np.expand_dims(x_test,axis=-1)

np.random.seed(42)
train_indices = np.arange(x_train.shape[0])
np.random.shuffle(train_indices)
x_train = x_train[train_indices]
y_train = y_train[train_indices]

In [ ]:
## for 18 channels 
adj=tf.constant(
    [  [1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.]],
    dtype=tf.float32)

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(physical_devices[1], 'GPU')

channel_names=["Fp1-T3","T3-O1","Fp1-C3","C3-O1","Fp2-C4","C4-O2","Fp2-T4","T4-O2","T3-C3","C3-Cz","Cz-C4","C4-T4"]
indices =[[r,i] for r,c1 in enumerate(channel_names) for i,c2 in enumerate(channel_names) if (c1.split("-")[0]==c2.split("-")[1] or c1.split("-")[1]==c2.split("-")[1] 
          or c1.split("-")[0]==c2.split("-")[0] or c1.split("-")[1]==c2.split("-")[0])]
adj=np.zeros((12,12))
for i in indices:
    adj[i[0]][i[1]]=1
adj=tf.constant(adj,dtype=tf.float32)

class GATLayer(layers.Layer):

    def __init__(self,output_dim):
        super(GATLayer, self).__init__()
        self.output_dim = output_dim
        self.Leakyrelu = layers.LeakyReLU(alpha=0.2)
    
    def build(self, input_shape):
        self.W = self.add_weight(name='W',shape=(input_shape[-1], self.output_dim), initializer='random_normal',trainable=True)
        self.a = self.add_weight(name='a',shape=(2*self.output_dim, 1), initializer='random_normal',trainable=True)
    
    def call(self,input,adj):
        H= tf.matmul(input, self.W)
        h1=tf.tile(tf.expand_dims(H, axis=1), [1,12,1,1])
        h2=tf.tile(tf.expand_dims(H, axis=2), [1,1,12,1])
        result =tf.concat([h1 , h2], axis=-1)
        e=self.Leakyrelu(tf.squeeze(tf.matmul(result, self.a),axis=-1))
        zero_mat= -1e20*tf.ones_like(e)
        msked_e=tf.where(adj==1.0,e,zero_mat)
        alpha=tf.nn.softmax(msked_e,axis=-1)
        HPrime=tf.matmul(alpha,H)
        return tf.nn.elu(HPrime)

def create_model():
    Input= keras.Input(shape=(12,384,1))
    regularizer_dense=regularizers.l2(0.0001)

    x= layers.Conv2D(32,(1,5),activation='relu',padding='same')(Input)
    y= layers.Conv2D(32,(1,7),activation='relu',padding='same')(Input)
    x= layers.add([x,y])
    x= layers.AveragePooling2D((1,2))(x)
    x= layers.BatchNormalization()(x)
    x= layers.SpatialDropout2D(0.2)(x)

    x= layers.Conv2D(64,(1,5),activation='relu',padding='same')(x)
    y= layers.Conv2D(64,(1,7),activation='relu',padding='same')(x)
    x= layers.add([x,y])
    x= layers.AveragePooling2D((1,2))(x)
    x= layers.BatchNormalization()(x)
    x= layers.SpatialDropout2D(0.2)(x)

    x= layers.Conv2D(8,(1,5),activation='relu',padding='same')(x)
    y= layers.SpatialDropout2D(0.2)(x)
    y= layers.Conv2D(8,(1,7),activation='relu',padding='same')(x)
    x= layers.add([x,y])
    x= layers.AveragePooling2D((1,2))(x)
    x= layers.BatchNormalization()(x)
    x= layers.SpatialDropout2D(0.2)(x)

    x= layers.Conv2D(1,(1,5),activation='relu',padding='same')(x)
    y= layers.SpatialDropout2D(0.2)(x)
    y= layers.Conv2D(1,(1,7),activation='relu',padding='same')(x)
    x= layers.add([x,y])
    x= layers.AveragePooling2D((1,2))(x)
    x= layers.Reshape((12,24))(x)

    x= GATLayer(37)(x,adj)
    x= GATLayer(32)(x,adj)
    x= GATLayer(16)(x,adj)

    x= layers.GlobalAveragePooling1D()(x)
    x= layers.Dropout(0.2)(x)
    x= layers.Dense(32,activation='relu',kernel_regularizer=regularizer_dense)(x)
    x= layers.Dropout(0.2)(x)
    x= layers.Dense(16,activation='relu',kernel_regularizer=regularizer_dense)(x)
    x= layers.Dropout(0.2)(x)
    x= layers.Dense(1,activation='sigmoid',kernel_regularizer=regularizer_dense)(x)

    model = keras.Model(inputs=Input, outputs=x)

    optimizer=keras.optimizers.Adam(learning_rate=0.002)
    loss=keras.losses.BinaryFocalCrossentropy(from_logits=False,gamma=2,alpha=0.4,apply_class_balancing=True)
    # loss=keras.losses.BinaryCrossentropy(from_logits=False)  
    kappa=tfa.metrics.CohenKappa(num_classes=2)
    fp=keras.metrics.FalsePositives()
    tn=keras.metrics.TrueNegatives()
    precision = keras.metrics.Precision()
    recall = keras.metrics.Recall()
    AUROC = keras.metrics.AUC(curve='ROC', name = 'AUROC')
    AUPRC = keras.metrics.AUC(curve='PR', name = 'AUPRC')
    model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy', AUROC, AUPRC,fp,tn, precision, recall,kappa])   
    return model
model=create_model()

In [ ]:
checkpoint_path = "GAT_correct_7/cp_{epoch:02d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path) 
cp_callback=keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=False,verbose=0,save_best_only=True,monitor='val_AUROC',mode='max')  
history=model.fit(x_train,y_train,epochs=50,batch_size=512,verbose=1,validation_data=(x_test,y_test),callbacks=[cp_callback])

In [ ]:
with open("./History/history_GAT_correct_6.jason", 'w') as f:
    pd.DataFrame(history.history).to_json(f)

In [ ]:
with open("./History/history_cv_0.jason", 'r') as f:
    history1=pd.read_json(f)
with open("./History/history_cv_1.jason", 'r') as f:
    history2=pd.read_json(f)
with open("./History/history_cv_2.jason", 'r') as f:
    history3=pd.read_json(f)
with open("./History/history_cv_3.jason", 'r') as f:
    history4=pd.read_json(f)
with open("./History/history_cv_4.jason", 'r') as f:
    history5=pd.read_json(f)
with open("./History/history_cv_5.jason", 'r') as f:
    history6=pd.read_json(f)
with open("./History/history_cv_6.jason", 'r') as f:
    history7=pd.read_json(f)
with open("./History/history_cv_7.jason", 'r') as f:
    history8=pd.read_json(f)
with open("./History/history_cv_8.jason", 'r') as f:
    history9=pd.read_json(f)
with open("./History/history_cv_9.jason", 'r') as f:
    history10=pd.read_json(f)

In [ ]:
# metrics=['accuracy','val_accuracy','loss','val_loss','val_AUROC','val_AUPRC','val_precision_3','val_recall_3']
# epochs = range(1, 101)
# fig,ax=plt.subplots(4,2,figsize=(20,20))
# for r in range(8):
#     ax[r//2][r%2].plot(epochs,history4[metrics[r]],label=metrics[r],color='r')
#     ax[r//2][r%2].set_title(metrics[r])
#     ax[r//2][r%2].set_xlabel('Epochs')
#     ax[r//2][r%2].grid()
# fig.tight_layout()
# plt.show()
print("Mean Accuracy: ",np.min([history1['val_accuracy'].max(),history2['val_accuracy'].max(),history3['val_accuracy'].max(),history4['val_accuracy'].max(),history5['val_accuracy'].max(),history6['val_accuracy'].max(),history7['val_accuracy'].max(),history8['val_accuracy'].max(),history9['val_accuracy'].max(),history10['val_accuracy'].max()]))
print("Mean AUROC: ",np.min([history1['val_AUROC'].max(),history2['val_AUROC'].max(),history3['val_AUROC'].max(),history4['val_AUROC'].max(),history5['val_AUROC'].max(),history6['val_AUROC'].max(),history7['val_AUROC'].max(),history8['val_AUROC'].max(),history9['val_AUROC'].max(),history10['val_AUROC'].max()]))
print("Mean AUPRC: ",np.min([history1['val_AUPRC'].max(),history2['val_AUPRC'].max(),history3['val_AUPRC'].max(),history4['val_AUPRC'].max(),history5['val_AUPRC'].max(),history6['val_AUPRC'].max(),history7['val_AUPRC'].max(),history8['val_AUPRC'].max(),history9['val_AUPRC'].max(),history10['val_AUPRC'].max()],))
print("Mean recall: ",np.min([history1['val_recall'].max(),history2['val_recall_1'].max(),history3['val_recall_2'].max(),history4['val_recall_3'].max(),history5['val_recall_4'].max(),history6['val_recall_5'].max(),history7['val_recall_6'].max(),history8['val_recall_7'].max(),history9['val_recall_8'].max(),history10['val_recall_9'].max()],))
print("Mean precision: ",np.min([history1['val_precision'].max(),history2['val_precision_1'].max(),history3['val_precision_2'].max(),history4['val_precision_3'].max(),history5['val_precision_4'].max(),history6['val_precision_5'].max(),history7['val_precision_6'].max(),history8['val_precision_7'].max(),history9['val_precision_8'].max(),history10['val_precision_9'].max()],))
print("Mean kappa: ",np.min([history1['val_cohen_kappa'].max(),history2['val_AUROC'].max(),history3['val_AUROC'].max(),history4['val_AUROC'].max(),history5['val_AUROC'].max(),history6['val_AUROC'].max(),history7['val_AUROC'].max(),history8['val_AUROC'].max(),history9['val_AUROC'].max(),history10['val_AUROC'].max()],))



In [ ]:
model1=keras.models.load_model('Saved_models/GAT_model_correct_4/cp_0053.ckpt')
model1.evaluate(x_test,y_test)  

In [ ]:
accuracy=[]
AUROC=[]
AUPRC=[]
recall=[]
precision=[]
kappa=[]
fp=[]
tn=[]

for i in range(1,40):
    with open(f"./History/history_cv_39_{i}.jason", 'r') as f:
        history=pd.read_json(f)
    if i==17 or i==1:
        accuracy.append(history['val_accuracy'].max())
        AUROC.append(history['val_AUROC'].max())
        AUPRC.append(history['val_AUPRC'].max())
        recall.append(history['val_recall'].max())
        precision.append(history['val_precision'].max())
        kappa.append(history['val_cohen_kappa'].max())
    else:
        if i>16:
            i=i-16
        accuracy.append(history['val_accuracy'].max())
        AUROC.append(history['val_AUROC'].max())
        AUPRC.append(history['val_AUPRC'].max())
        recall.append(history[f'val_recall_{i-1}'].max())
        precision.append(history[f'val_precision_{i-1}'].max())
        kappa.append(history['val_cohen_kappa'].max())

print("Mean Accuracy: ",np.mean(np.array(accuracy)))
print("Mean AUROC: ",np.mean(np.array(AUROC)))
print("Mean AUPRC: ",np.mean(np.array(AUPRC)))
print("Mean recall: ",np.mean(np.array(recall)))
print("Mean precision: ",np.mean(np.array(precision)))
print("Mean kappa: ",np.mean(np.array(kappa)))
